<img src="https://files.cdn.thinkific.com/file_uploads/133292/images/d57/1b0/1a9/P03.jpg">

`MCSDSC02P03V3______`

`MCSDSELIMC02P03V2250621`

`MCSDSELIMC02P03V1130621`

## Постановка задачи

Выполнить анализ заказчика - страховой компании.

Задача:
- проанализировать выплаты страховых компенсаций;
- определить как разные страховые случаи, возраст клиента и его доход влияют на статус выплаты страховой компенсации.

## Импорт данных и библиотек

Импортируем библиотеки, прочитаем файл `insurance.csv` и получим общую информацию о данных.

In [1]:
import pandas as pd
import numpy as np

insurance_data = pd.read_csv('/Users/valli/Desktop/mcs_ds_02_upd/data/insurance.csv')
insurance_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2301 entries, 0 to 2300
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              2301 non-null   int64  
 1   perc_of_compensation_paid       2301 non-null   float64
 2   age_in_days                     2301 non-null   float64
 3   age_id                          2301 non-null   int64  
 4   age_group                       2301 non-null   object 
 5   income                          2236 non-null   float64
 6   count_3-6_months_late           2301 non-null   int64  
 7   count_6-12_months_late          2301 non-null   int64  
 8   count_more_than_12_months_late  2301 non-null   int64  
 9   cause                           2301 non-null   object 
dtypes: float64(3), int64(5), object(2)
memory usage: 179.9+ KB


Столбцы содержат следующую информацию:

- идентификиционный номер клиента;
- процент выплаченной компенсации;
- возраст (кол-во прожитых дней);
- возрастная категория;
- возрастная группа;
- доход;
- страховка оформлена не позднее 3-6 месяцев;
- страховка оформлена не позднее 6-12 месяцев;
- страховка оформлена больше года назад;
- причина страховой выплаты.


### Выводы

* Данные содержат информация о клинте (застрахованном лице), его возрасте, доходе, статусе стаховой выплаты и её причина (страховой случай)

* Датасает состоит из 10 столбцов и 2301 строк, среди них три строки с типом данных float64, пять -  int64, две - object

## Предобработка данных

**Обработка пропусков**

Проверим данные на наличие пропусков, просуммировав их.

In [2]:
insurance_data.isna().sum()

id                                 0
perc_of_compensation_paid          0
age_in_days                        0
age_id                             0
age_group                          0
income                            65
count_3-6_months_late              0
count_6-12_months_late             0
count_more_than_12_months_late     0
cause                              0
dtype: int64

Заполним пропуски в столбце `income`.

In [3]:
insurance_data['income'] = insurance_data['income'].fillna('0000')

Проверяем, что пропусков в `income` не осталось.

In [4]:
insurance_data.isna().sum()

id                                0
perc_of_compensation_paid         0
age_in_days                       0
age_id                            0
age_group                         0
income                            0
count_3-6_months_late             0
count_6-12_months_late            0
count_more_than_12_months_late    0
cause                             0
dtype: int64

**Замена типа данных**

Проверяем, что в столбцах `age_in_days` и `income` вещественный тип данных.

In [5]:
insurance_data[['age_in_days','income']].dtypes

age_in_days    float64
income          object
dtype: object

Для этих столбцов нужен целочисленный тип, поэтому переведем их, а также избавимся от отрицательных значений в `income`.

In [6]:
insurance_data[['age_in_days','income']] = insurance_data[['age_in_days','income']].astype('int64')

In [7]:
insurance_data['income'] = insurance_data['income'].abs()

**Поиск дубликатов с учетом регистра**

Проверим уникальные категории в столбце `age_group`.

In [8]:
insurance_data['age_group'].unique()

array(['Пожилой', 'пожилой', 'Зрелый', 'Молодой', 'ЗРЕЛЫЙ', 'ПОЖИЛОЙ',
       'зрелый', 'молодой', 'МОЛОДОЙ'], dtype=object)

Приведем их к нижнему регистру.

In [9]:
insurance_data['age_group'] = insurance_data['age_group'].str.lower()

**Обработка дубликатов**

Установим количество явных дубликатов. Если найдутся, то удалим и снова проверим, что их не осталось.

In [10]:
insurance_data.duplicated().sum()

7

In [11]:
insurance_data = insurance_data.drop_duplicates().reset_index(drop=True)

In [12]:
insurance_data.duplicated().sum()

0

**Стэмминг**

Посчитаем уникальные значения в столбце `cause`.

In [13]:
insurance_data['cause'].value_counts()

кража автомобиля               112
травма во время отдыха         107
сломала ногу в отпуске          95
упало дерево на автомобиль      94
украли багаж в отпуске          94
травма в отпуске                94
сгорел дом                      93
замкнуло электричество дома     91
сгорела квартира                91
врезались в автомобиль          91
несчастный случай в отпуске     91
сотрясение во время отдыха      90
проникли в жилье                89
повреждение автомобиля          89
потоп в квартире                88
хищение автомобиля (каско)      88
несчастный случай на отдыхе     85
повреждение авто                85
кража авто                      82
обокрали дачу                   81
поломка авто                    81
кража в квартире                79
украли авто                     78
взломали квартиру               77
дтп                             76
затопило дом                    73
Name: cause, dtype: int64

Выделим очевидные разные стеммы:

* авто
* квартира
* дача
* дом
* отдых
* дтп

Из них можно выделить следующие виды страхования:

страхование автотранспорта, страхование недвижимости, страхование во время отпуска.

Напишем функцию, которая принимает неизменяемую часть слова и возвращает все названия категорий с ним. Сначала попробуем стемминг и если не поймаем «чужих» названий — значит этот метод подойдет.

In [14]:
def specific_cause(df, column, word):
    filter_cause = df.loc[df[column].str.contains(word)]
    result = filter_cause[column].value_counts()
    return result

In [15]:
specific_cause(insurance_data, 'cause', 'авто')

кража автомобиля              112
упало дерево на автомобиль     94
врезались в автомобиль         91
повреждение автомобиля         89
хищение автомобиля (каско)     88
повреждение авто               85
кража авто                     82
поломка авто                   81
украли авто                    78
Name: cause, dtype: int64

**Категоризация данных**

Cловарь — распространенный способ хранения информации, который помогает убрать текстовые параметры из основной таблицы и увеличить эффективность работы с данными. Выделим словарь для возрастных групп.

In [16]:
age_group_df = insurance_data[['age_group']].copy()
age_group_df.head(10)

,age_group
0,пожилой
1,пожилой
2,зрелый
3,зрелый
4,молодой
5,зрелый
6,зрелый
7,пожилой
8,пожилой
9,пожилой


Категоризируем причины выплаты страховых платежей с помощью лемматизации.

In [17]:
from pymystem3 import Mystem
m = Mystem()

def lem(row):
    l = m.lemmatize(row)
    if 'автомобиля' in l or 'автомобиль' in l or 'авто' in l or 'дтп' in l:
        return 'Автотранспорт'
    elif 'дом' in l or 'квартира' in l or 'квартире' in l or 'квартиру' in l or 'жилье' in l or 'дачу' in l:
        return 'Недвижимость'
    elif 'отпуске' in l or 'отдыхе' in l or 'отпуск' in l or 'отдыха' in l:
        return 'Отпуск'
    
insurance_data['categorized cause'] = insurance_data['cause'].apply(lem)

In [18]:
insurance_data.tail(10)
#не понимаю почему он не считывает "отдыха" или "дачу" и ставит None в категории

,id,perc_of_compensation_paid,age_in_days,age_id,age_group,income,count_3-6_months_late,count_6-12_months_late,count_more_than_12_months_late,cause,categorized cause
2284,65351,0.229,22638,4,пожилой,0,0,0,0,кража автомобиля,Автотранспорт
2285,19888,0.041,10598,2,молодой,0,0,0,0,кража автомобиля,Автотранспорт
2286,61783,0.824,10965,2,молодой,0,0,0,0,кража в квартире,Недвижимость
2287,58158,0.083,12790,3,зрелый,0,0,0,0,проникли в жилье,Недвижимость
2288,36067,0.012,25190,4,пожилой,0,0,0,0,затопило дом,Недвижимость
2289,16440,0.474,18628,3,зрелый,0,0,0,0,дтп,Автотранспорт
2290,109416,0.413,17525,3,зрелый,0,0,0,0,кража в квартире,Недвижимость
2291,4268,0.109,21906,4,пожилой,0,0,0,0,проникли в жилье,Недвижимость
2292,85708,0.036,25925,4,пожилой,0,0,0,0,обокрали дачу,None
2293,105186,0.010,23370,4,пожилой,0,0,0,0,повреждение автомобиля,Автотранспорт


Категоризируем статус выплаты компенсации — 0 выплат, выплачена часть, полностью выплачена.

In [19]:
conditions = [
     insurance_data['perc_of_compensation_paid'] == 0.000,
     insurance_data['perc_of_compensation_paid'] == 1.000,
    (insurance_data['perc_of_compensation_paid'] > 0.000) & (insurance_data['perc_of_compensation_paid'] < 1.000)
     ]

values = ['0 выплат', 'полностью выплачена', 'выплачена часть']

insurance_data['categorized payments'] = np.select(conditions, values)

Категоризируем `income`, разбив данные по квантилям.

In [20]:
insurance_data['categorized by quantile']=pd.qcut(insurance_data['income'],5, precision=0)

In [23]:
insurance_data.head(20)

,id,perc_of_compensation_paid,age_in_days,age_id,age_group,income,count_3-6_months_late,count_6-12_months_late,count_more_than_12_months_late,cause,categorized cause,categorized payments,categorized by quantile
0,649,0.001,27384,4,пожилой,0,0,0,0,повреждение автомобиля,Автотранспорт,выплачена часть,"(-1.0, 90060.0]"
1,81136,0.124,23735,4,пожилой,285140,0,0,0,упало дерево на автомобиль,Автотранспорт,выплачена часть,"(269868.0, 2700040.0]"
2,70762,1.000,17170,3,зрелый,186030,0,0,0,сломала ногу в отпуске,Отпуск,полностью выплачена,"(135758.0, 189668.0]"
3,53935,0.198,16068,3,зрелый,123540,0,0,0,хищение автомобиля (каско),Автотранспорт,выплачена часть,"(90060.0, 135758.0]"
4,15476,0.041,10591,2,молодой,200020,1,0,0,несчастный случай в отпуске,Отпуск,выплачена часть,"(189668.0, 269868.0]"
5,64797,0.112,16065,3,зрелый,0,0,0,0,врезались в автомобиль,Автотранспорт,выплачена часть,"(-1.0, 90060.0]"
6,67412,0.325,17167,3,зрелый,388050,6,0,0,травма во время отдыха,None,выплачена часть,"(269868.0, 2700040.0]"
7,44241,0.362,20085,4,пожилой,99880,3,0,0,повреждение автомобиля,Автотранспорт,выплачена часть,"(90060.0, 135758.0]"
8,5069,0.115,23008,4,пожилой,360040,0,0,0,сгорела квартира,Недвижимость,выплачена часть,"(269868.0, 2700040.0]"
9,16615,0.021,21906,4,пожилой,510040,0,0,0,сгорел дом,Недвижимость,выплачена часть,"(269868.0, 2700040.0]"


## Результаты

Подготовим 3 таблицы и изучим результат.

*ответьте на вопрос — есть ли зависимость между возрастной группой и статусом выплаты?*

In [29]:
compensation_by_age = insurance_data.pivot_table(index=['categorized payments','age_group'], values='id', aggfunc='count').transpose()
compensation_by_age

categorized payments 0 выплат                 выплачена часть                  \
age_group              зрелый молодой пожилой          зрелый молодой пожилой   
id                         62      31      57             884     206     698   

categorized payments полностью выплачена                  
age_group                         зрелый молодой пожилой  
id                                   100      52     204

Судя по данным таблицы полностью страховые выплаты чаще всего выплачиваются клиентам в пожилой возрастной категории и меньше всего клиентам в молодой возрастной категории.

Частичная выплата чаще всего выплачивается клиентам в зрелой возрастной категории и меньше всего клиентам в молодой возрастной категории.

Не выплачивают совсем чаще всего клиентам в зрелой возрастной категории и реже всего - молодым клиентам.


*есть ли зависимость между причиной страхового случая и статусом выплаты?*

In [31]:
compensation_by_cause = insurance_data.pivot_table(index=['categorized payments','categorized cause'], values='id', aggfunc='count').transpose()
compensation_by_cause

categorized payments      0 выплат                     выплачена часть  \
categorized cause    Автотранспорт Недвижимость Отпуск   Автотранспорт   
id                              54           49     25             640   

categorized payments                     полностью выплачена               \
categorized cause    Недвижимость Отпуск       Автотранспорт Недвижимость   
id                            548    308                 182           84   

categorized payments         
categorized cause    Отпуск  
id                       41

Судя по данным таблицы полностью страховые выплаты чаще всего выплачиваются за стаховой случай с автотранспортом, меньше всего за отпуск.

Частичная выплата чаще всего выплачивается также за страховой случай с автотранспортом и меньше всего за отпуск.

Не выплачивают совсем чаще всего за страховой случай с автотранспортном и реже всего - за отпуск.

*есть ли зависимость между доходом клиента и статусом выплаты?*

In [34]:
compensation_by_income = insurance_data.pivot_table(index=['categorized payments','categorized by quantile'], values='id', aggfunc='count').transpose()
compensation_by_income

categorized payments           0 выплат                      \
categorized by quantile (-1.0, 90060.0] (90060.0, 135758.0]   
id                                   43                  30   

categorized payments                                               \
categorized by quantile (135758.0, 189668.0] (189668.0, 269868.0]   
id                                        33                   19   

categorized payments                          выплачена часть  \
categorized by quantile (269868.0, 2700040.0] (-1.0, 90060.0]   
id                                         25             326   

categorized payments                                              \
categorized by quantile (90060.0, 135758.0] (135758.0, 189668.0]   
id                                      356                  366   

categorized payments                                                \
categorized by quantile (189668.0, 269868.0] (269868.0, 2700040.0]   
id                                       371                   369   

categorized payments    полностью выплачена                      \
categorized by quantile     (-1.0, 90060.0] (90060.0, 135758.0]   
id                                       94                  69   

categorized payments                                               \
categorized by quantile (135758.0, 189668.0] (189668.0, 269868.0]   
id                                        59                   69   

categorized payments                           
categorized by quantile (269868.0, 2700040.0]  
id                                         65

Судя по данным таблицы полностью страховые выплаты чаще всего выплачиваются клиентам с самым низким доходом, реже всего клиентам с самым высоким доходом.

Частичная выплата чаще всего выплачивается клиентам с самым высоким доходом и реже всего с самым низким.

Не выплачивают совсем чаще всего клиентам с самым низким доходом, а реже всего - клиентам с самым высоким.